In [ ]:
import metpy
import datetime
import rioxarray
import os
import hvplot.xarray
import hvplot.pandas
import hvplot
import glob
import warnings

import geopandas as gpd
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.feature as cfeature

from pyproj import Proj
from tqdm.notebook import tqdm
from shapely.geometry import mapping
from dask import delayed
from dask.distributed import Client
from holoviews import opts

from plotting import histogram_density_plot

In [ ]:
dates = pd.date_range(start='2021-01-01', end='2022-01-01', freq='1d', inclusive="left", tz='GMT').values
files = {}
for i in dates:
    date = f'{pd.to_datetime(i).dayofyear:03}'
    files[date] = sorted(glob.glob(f'/Volumes/Shores/GOES/AODC/averages/2021/{date}/*.nc'))

In [ ]:
aod = xr.open_dataset(
    files['001'][0]
)

In [ ]:
geo = ccrs.Geostationary(
    central_longitude=aod.goes_imager_projection.longitude_of_projection_origin,
    satellite_height=aod.goes_imager_projection.perspective_point_height,
    sweep_axis=aod.goes_imager_projection.sweep_angle_axis
)

In [ ]:
plate = ccrs.PlateCarree()

In [6]:
for k, vs in files.items():
    ds = xr.open_mfdataset(
        vs,
        concat_dim="t", 
        parallel=True, 
        combine='nested',
        data_vars='different',
    ).compute()
    sat_h = ds.goes_imager_projection.perspective_point_height
    ds = ds.assign_coords(x=ds.x*sat_h, y = ds.y*sat_h)
    
    ds2 = xr.open_dataset(f'/Volumes/Shores/EPA/daily_epa/{k}.nc')
    ds2 = ds2.rename_dims({'time': 't', 'latitude': 'y', 'longitude': 'x'})
    ds2 = ds2.where(ds2.time.isin(ds.t), drop=True)
    
    # transpose to make the coordinates align
    ds['PM2.5'] = ds2['PM2.5'].transpose()
    
    df = ds.drop_vars(['y_image',
     'x_image',
     'sunglint_angle',
     'retrieval_local_zenith_angle',
     'quantitative_local_zenith_angle',
     'retrieval_solar_zenith_angle',
     'quantitative_solar_zenith_angle',
     'aod_product_wavelength'])[['AOD', 'PM2.5']]\
        .to_dataframe().reset_index().drop(['t', 'y', 'x'], axis=1).dropna()
    
    latlon = plate.transform_points(src_crs=geo, x=df['longitude'], y=df['latitude'])
    df['longitude'] = latlon[:,0]
    df['latitude'] = latlon[:,1]
    df.to_csv(f'/Volumes/Shores/Matched/{k}.csv')


KeyboardInterrupt

